In [1]:
from pikov import pikov
gamekitty = pikov.Pikov.open('gamekitty.pikov')

In [32]:
import math
import networkx.drawing.layout
import IPython.display

In [2]:
gamekitty

In [3]:
stand_waggle = gamekitty.get_frame('stand_waggle_0')
stand_waggle

In [4]:
stand_waggle.transitions[0]

In [5]:
stand_waggle.transitions[0].target

In [6]:
gamekitty.get_image('md5-9b084a2f4cc7a7f940fac47ebe710c45')

In [7]:
#sit = gamekitty.get_clip('sit')
#sit_to_stand = gamekitty.get_clip('sit_to_stand')
#sit_paw = gamekitty.get_clip('sit_paw')
#stand_to_sit = gamekitty.get_clip('stand_to_sit')
#sit + sit_to_stand + clip + clip + stand_to_sit + sit_paw

In [8]:
frames = list(gamekitty.list_frames())
transitions = list(gamekitty.list_transitions())
graph = gamekitty.to_networkx()

In [4]:
layout = networkx.drawing.layout.spring_layout(graph)
layout

{Frame(id='crouch_0'): array([0.75777248, 0.35427322]),
 Frame(id='crouch_to_sit_0'): array([0.48278861, 0.21355126]),
 Frame(id='crouch_to_sit_1'): array([0.35480132, 0.01012861]),
 Frame(id='crouch_to_sit_2'): array([0.15898066, 0.06696738]),
 Frame(id='lie_down_0'): array([1.        , 0.40333179]),
 Frame(id='sit_0'): array([-0.00401823,  0.09807254]),
 Frame(id='sit_paw_0'): array([-0.46702901, -0.06259738]),
 Frame(id='sit_paw_1'): array([-0.33022163,  0.20011679]),
 Frame(id='sit_paw_2'): array([-0.11011247,  0.39384272]),
 Frame(id='sit_paw_3'): array([0.07950145, 0.30774954]),
 Frame(id='sit_to_crouch_0'): array([ 0.58705968, -0.42675425]),
 Frame(id='sit_to_crouch_1'): array([ 0.8425669 , -0.08617257]),
 Frame(id='sit_to_crouch_2'): array([0.94615808, 0.30193299]),
 Frame(id='sit_to_stand_0'): array([-0.33754812,  0.75536807]),
 Frame(id='sit_to_stand_1'): array([-0.47729646,  0.4995863 ]),
 Frame(id='sit_to_stand_2'): array([-0.73566226,  0.22291604]),
 Frame(id='sit_to_stand

In [78]:
LINE_WIDTH = 0.01
LINE_COLOR = '#83769c'  # 'lightblue'
IMAGE_SIZE = 0.1
ARROW_WIDTH = LINE_WIDTH * 4
ARROW_POINT = ARROW_WIDTH / 3.0

# Radius to the corner of the image.
IMAGE_RADIUS = (IMAGE_SIZE / 2.0) * math.sqrt(2.0)
IMAGE_RADIUS *= 1.05  # A little slush.


def arrow_head(x, y, angle):
    x_start = x + (
        -1.0 * math.cos(angle) * ARROW_POINT +
        -1.0 * math.sin(angle) * ARROW_WIDTH / 2.0
    )
    x_end = x + (
        -1.0 * math.cos(angle) * ARROW_POINT +
        math.sin(angle) * ARROW_WIDTH / 2.0
    )
    y_start = y + (
        math.cos(angle) * ARROW_WIDTH / 2.0 +
        -1.0 * math.sin(angle) * ARROW_POINT
    )
    y_end = y + (
        -1.0 * math.cos(angle) * ARROW_WIDTH / 2.0 +
        -1.0 * math.sin(angle) * ARROW_POINT
    )
    return (
        f'<path d="M {x_start} {y_start} L {x} {y} {x_end} {y_end}" '
        f'stroke="{LINE_COLOR}" stroke-width="{LINE_WIDTH}" fill="none" />'
    )


def to_svg(layout, frames, transitions):
    lines = []
    for transition in transitions:
        if transition.source != transition.target:
            x1, y1 = layout[transition.source]
            x2, y2 = layout[transition.target]
            angle = math.atan2(y2 - y1, x2 - x1)
            xo = math.cos(angle) * IMAGE_RADIUS
            yo = math.sin(angle) * IMAGE_RADIUS
            line = (
                f'<line x1="{x1 + xo}" y1="{y1 + yo}" x2="{x2 - xo}" y2="{y2 - yo}" '
                f'stroke="{LINE_COLOR}" stroke-width="{LINE_WIDTH}" />'
            )
            line += arrow_head(x2 - xo, y2 - yo, angle)
        else:
            x, y = layout[transition.source]
            line = (
                f'<path d="M {x} {y - IMAGE_RADIUS} '
                f'A {IMAGE_RADIUS} {IMAGE_RADIUS} 0 0 0 {x - 2 * IMAGE_RADIUS} {y - IMAGE_RADIUS} '
                f'{IMAGE_RADIUS} {IMAGE_RADIUS} 0 0 0 {x - IMAGE_RADIUS} {y}" '
                f'stroke="{LINE_COLOR}" stroke-width="{LINE_WIDTH}" fill="none" />'
            )
            line += arrow_head(x - IMAGE_RADIUS, y, 0)
        lines.append(line)
    images = []
    for frame in frames:
        point = layout[frame]
        x = point[0] - 0.05
        y = point[1] - 0.05
        imgurl = frame.image._to_data_url()
        image = (
            f'<image href="{imgurl}" image-rendering="optimizeSpeed" '
            f'x="{x}" y="{y}" height="{IMAGE_SIZE}" width="{IMAGE_SIZE}" />'
        )
        images.append(image)
    linesstr = '\n'.join(lines)
    imagesstr = '\n'.join(images)
    return f"""
    <svg viewBox="-1.1 -1.1 2.2 2.2" xmlns="http://www.w3.org/2000/svg">
    {linesstr}
    {imagesstr}
    </svg>
    """

In [79]:
layout = networkx.drawing.layout.spring_layout(graph)
IPython.display.display(IPython.display.SVG(data=to_svg(layout, frames, transitions)))

In [80]:
circular_layout = networkx.drawing.layout.circular_layout(graph)
IPython.display.display(IPython.display.SVG(data=to_svg(circular_layout, frames, transitions)))

In [81]:
kamada_kawai_layout = networkx.drawing.layout.kamada_kawai_layout(graph)
IPython.display.display(IPython.display.SVG(data=to_svg(kamada_kawai_layout, frames, transitions)))

In [82]:
shell_layout = networkx.drawing.layout.shell_layout(graph)
IPython.display.display(IPython.display.SVG(data=to_svg(shell_layout, frames, transitions)))

In [83]:
spectral_layout = networkx.drawing.layout.spectral_layout(graph)
IPython.display.display(IPython.display.SVG(data=to_svg(spectral_layout, frames, transitions)))